Nova tentativa


Mudança na verificação dos contours, porque em alguns casos a countour do tabuleiro não é representado como um quadrilatero. :(

In [28]:
import cv2
import numpy as np



def print_resized(name, img):
    resize_img = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
    cv2.imshow(name, resize_img)



def detect_game_board(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not load image.")
        return None, None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0) # to remove some noise
    edges = cv2.Canny(blurred, 50, 150)
    kernel = np.ones((3, 3), np.uint8)

    print_resized("Canny Edges", edges)

    dilated_edges = cv2.dilate(edges, kernel, iterations=1) # dilate to make them more visible

    print_resized("Canny Dilated", dilated_edges)


    kernel = np.ones((7, 7), np.uint8)  # Larger kernel for better connection
    closed = cv2.morphologyEx(dilated_edges, cv2.MORPH_CLOSE, kernel, iterations=3)

    print_resized("Closed", closed)

    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    board_contour = None
    max_area = 0
    i = 0
    for contour in contours:
        # Get convex hull because it works better

        hull = cv2.convexHull(contour)
        peri = cv2.arcLength(hull, True)
        approx = cv2.approxPolyDP(hull, 0.05 * peri, True)  # Higher tolerance for better board detection
        
        # If is has 4 sides
        if len(approx) == 4:
            i += 1
            print("Contour" + str(i))
            area = cv2.contourArea(contour)
            if area > max_area:
                max_area = area
                board_contour = approx
                

    if board_contour is None:
        print("No game board detected.")
        return image, None

    corners = board_contour.reshape(4, 2).astype(np.float32) #get the corners of the board
    width, height = 500, 500
    dst_points = np.array([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]], dtype=np.float32)
    matrix = cv2.getPerspectiveTransform(corners, dst_points)
    warped = cv2.warpPerspective(image, matrix, (width, height))

    cv2.drawContours(image, [board_contour], -1, (0, 255, 0), 3)
    return image, warped


# For testing
image_path = "images\G000_IMG062.jpg"
image_path = "images\G000_IMG087.jpg"
image_path = "images\G000_IMG102.jpg"
image_path = "images\G006_IMG048.jpg"
#image_path = "images\G006_IMG086.jpg"
#image_path = "images\G006_IMG119.jpg"
#image_path = "images\G019_IMG082.jpg"
#image_path = "images\G028_IMG015.jpg" #aqui muito fora ESTE É MUITO COMPLEXO XD
#image_path = "images\G028_IMG062.jpg"
image_path = "images\G028_IMG098.jpg" #aqui ligeiramente
#image_path = "images\G028_IMG101.jpg"
#image_path = "images\G033_IMG043.jpg"
#image_path = "images\G033_IMG075.jpg"
#image_path = "images\G033_IMG088.jpg"
#image_path = "images\G033_IMG101.jpg"
#image_path = "images\G038_IMG074.jpg"
#image_path = "images\G038_IMG088.jpg"
#image_path = "images\G038_IMG103.jpg"
image_path = "images\G038_IMG105.jpg" #mais para dentro
#image_path = "images\G041_IMG042.jpg"
#image_path = "images\G041_IMG048.jpg"
#image_path = "images\G041_IMG088.jpg"
#image_path = "images\G041_IMG098.jpg"
#image_path = "images\G047_IMG053.jpg"
#image_path = "images\G047_IMG068.jpg"
#image_path = "images\G047_IMG102.jpg"
image_path = "images\G047_IMG107.jpg" #aqui ligeiramente
#image_path = "images\G056_IMG017.jpg"
image_path = "images\G056_IMG077.jpg" #aqui ligeiramente
image_path = "images\G056_IMG097.jpg" #aqui ligeiramente
#image_path = "images\G058_IMG044.jpg"
#image_path = "images\G058_IMG074.jpg"
#image_path = "images\G058_IMG100.jpg"
image_path = "images\G061_IMG080.jpg" #mais para dentro
#image_path = "images\G061_IMG092.jpg"
#image_path = "images\G061_IMG098.jpg"
image_path = "images\G072_IMG083.jpg" #aqui
#image_path = "images\G072_IMG098.jpg"
#image_path = "images\G076_IMG072.jpg"
#image_path = "images\G076_IMG089.jpg"
#image_path = "images\G076_IMG095.jpg"
#image_path = "images\G078_IMG092.jpg"
#image_path = "images\G083_IMG073.jpg"
#image_path = "images\G083_IMG089.jpg"
image_path = "images\G087_IMG093.jpg" #aqui
#image_path = "images\G087_IMG099.jpg"
#image_path = "images\G091_IMG053.jpg"
#image_path = "images\G091_IMG074.jpg"
#image_path = "images\G091_IMG102.jpg"
#image_path = "images\G099_IMG094.jpg"



img_copy = cv2.imread(image_path)
print_resized("Original Image", img_copy)

original_with_contour, corrected_board = detect_game_board(image_path)
if original_with_contour is not None:
    cv2.imshow("Original with Contour", original_with_contour)
    if corrected_board is not None:
        cv2.imshow("Corrected Board", corrected_board)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Contour1
Contour2
Contour3
Contour4
Contour5


Ler o cavalinho, (Foi a função que se usou para obter a imagem do cavalinho) Não interessa para a entrega

In [30]:
def interactive_image_editor(image_path):
    """
    Interactive image viewer and editor with rotation and ROI selection capabilities.
    
    Controls:
    - 'l': Rotate left (counter-clockwise) by 5 degrees
    - 'r': Rotate right (clockwise) by 5 degrees
    - 's': Select ROI and save it
    - 'q': Quit
    
    Args:
        image_path (str): Path to the image file
    """
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return
    
    img = corrected_board

    # Create a window
    window_name = "Interactive Image Editor"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    
    # Calculate appropriate display size
    screen_width = 1280  # Reasonable default screen width
    screen_height = 720  # Reasonable default screen height
    
    img_height, img_width = img.shape[:2]
    scale = min(screen_width / img_width, screen_height / img_width * 0.8)
    
    # Set window size
    display_width = int(img_width * scale)
    display_height = int(img_height * scale)
    cv2.resizeWindow(window_name, display_width, display_height)
    
    # Keep track of rotation angle
    angle = 0
    rotation_step = 5  # degrees per key press
    
    # Make a copy of the original image
    current_img = img.copy()
    
    while True:
        # Display the current image
        cv2.imshow(window_name, current_img)
        
        # Wait for key press
        key = cv2.waitKey(0) & 0xFF
        
        # Handle key presses
        if key == ord('q'):
            # Quit
            break
            
        elif key == ord('l'):
            # Rotate left (counter-clockwise)
            angle += rotation_step
            rows, cols = img.shape[:2]
            M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
            current_img = cv2.warpAffine(img, M, (cols, rows))
            print(f"Rotated left to {angle} degrees")
            
        elif key == ord('r'):
            # Rotate right (clockwise)
            angle -= rotation_step
            rows, cols = img.shape[:2]
            M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
            current_img = cv2.warpAffine(img, M, (cols, rows))
            print(f"Rotated right to {angle} degrees")
            
        elif key == ord('s'):
            # Enter ROI selection mode
            print("Select a region of interest (ROI) and press ENTER when done")
            roi = cv2.selectROI(window_name, current_img, fromCenter=False, showCrosshair=True)
            
            if roi != (0, 0, 0, 0):
                # Extract the ROI
                x, y, w, h = roi
                roi_img = current_img[y:y+h, x:x+w]
                
                # Save the ROI
                output_path = f"roi_{angle}deg_{x}_{y}_{w}x{h}.jpg"
                cv2.imwrite(output_path, roi_img)
                print(f"ROI saved to {output_path}")
                
                # Show the ROI
                cv2.imshow("Selected ROI", roi_img)
    
    cv2.destroyAllWindows()

# Example usage:
interactive_image_editor("images/G000_IMG062.jpg")

Rotated left to 5 degrees
Rotated left to 10 degrees
Rotated left to 15 degrees
Rotated left to 20 degrees
Rotated left to 25 degrees
Rotated left to 30 degrees
Rotated left to 35 degrees
Rotated left to 40 degrees
Rotated left to 45 degrees
Rotated left to 50 degrees
Rotated left to 55 degrees
Rotated left to 60 degrees
Rotated left to 65 degrees
Rotated left to 70 degrees
Rotated left to 75 degrees
Rotated left to 80 degrees
Rotated left to 85 degrees
Rotated left to 90 degrees
Rotated left to 95 degrees
Rotated left to 100 degrees
Rotated left to 105 degrees
Rotated left to 110 degrees
Rotated left to 115 degrees
Rotated left to 120 degrees
Rotated left to 125 degrees
Rotated left to 130 degrees
Rotated left to 135 degrees
Rotated left to 140 degrees
Rotated left to 145 degrees
Rotated left to 150 degrees
Rotated left to 155 degrees
Rotated left to 160 degrees
Rotated left to 165 degrees
Rotated left to 170 degrees
Rotated left to 175 degrees
Rotated left to 180 degrees
Select a reg

Ver o cavalinho

In [16]:
import cv2
import numpy as np


img1 = cv2.imread('cavalinhoPequeno.jpg', cv2.IMREAD_GRAYSCALE) 
#img2 = cv2.imread('images\G000_IMG102.jpg', cv2.IMREAD_GRAYSCALE)
#img2 = cv2.imread('images\G000_IMG087.jpg', cv2.IMREAD_GRAYSCALE)
#img2 = cv2.imread('images\G000_IMG062.jpg', cv2.IMREAD_GRAYSCALE)
#img2 = cv2.imread('images\G006_IMG048.jpg', cv2.IMREAD_GRAYSCALE)
#img2 = corrected_board
#img2 = corrected_board


img2 = cv2.cvtColor(corrected_board, cv2.COLOR_BGR2GRAY)

#Preprocess for SIFT
img1 = cv2.equalizeHist(img1)
img2 = cv2.equalizeHist(img2)

# Initiate SIFT detector
sift = cv2.SIFT_create()

# Find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# Apply FLANN Matcher
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Store all matches (taking the best match from each pair)
good = [m for m, n in matches]

# Prepare points for homography
query_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)  # img1
train_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)  # img2

# Find homography from query (img1) to train (img2)
M, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0)

# Define corners of the query image
h, w = img1.shape[:2]


corners = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)


# Transform corners to train image coordinates
transformed_corners = cv2.perspectiveTransform(corners, M)


# Create a copy of train image for drawing
train_with_box = img2.copy()

# Draw lines connecting the transformed corners
transformed_corners = np.int32(transformed_corners)
cv2.polylines(train_with_box, [transformed_corners], True, (0, 255, 255), 2)




# Display results
cv2.imshow('Query Image', img1)
cv2.imshow('Train Image with Object Outline', train_with_box)


#To Guarantee angle og 0, 90, 180 or 270 degrees
def extract_rotation_from_homography(M):
    # Extract rotation angle from homography matrix
    a = M[0, 0]
    b = M[0, 1]
    c = M[1, 0]
    d = M[1, 1]
    
    # Calculate rotation angle
    angle_rad = np.arctan2(c, a)
    raw_angle_deg = np.degrees(angle_rad)
    
    # Normalize angle to 0-360 range
    raw_angle_deg = raw_angle_deg % 360
    if raw_angle_deg < 0:
        raw_angle_deg += 360
        
    # Quantize to 0, 90, 180, or 270 degrees
    quantized_angle = round(raw_angle_deg / 90) * 90
    if quantized_angle == 360:
        quantized_angle = 0
        
    print(f"Raw rotation angle: {raw_angle_deg:.2f} degrees")
    print(f"Quantized rotation angle: {quantized_angle:.0f} degrees")
    return quantized_angle



# Extract rotation angle from homography matrix M
rotation_angle = extract_rotation_from_homography(M)

# Create a pure rotation matrix centered on the image
h, w = img2.shape[:2]
center = (w // 2, h // 2)
rotation_matrix = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)

# Apply the rotation to the train image
rotated_img = cv2.warpAffine(corrected_board, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)

# Display the results
cv2.imshow('Original Train Image', img2)
cv2.imshow('Rotated Train Image (center fixed)', rotated_img)

# Draw crosshair at center point for visualization
center_img = img2.copy()
cv2.drawMarker(center_img, center, (255, 255, 255), cv2.MARKER_CROSS, 20, 2)
cv2.imshow('Center Point', center_img)


corrected_board = rotated_img

cv2.waitKey(0)
cv2.destroyAllWindows()

Raw rotation angle: 182.71 degrees
Quantized rotation angle: 180 degrees


In [17]:
# Try for the hough lines

corrected_board2 = corrected_board.copy()

cv2.imshow("board", corrected_board)
#print_resized("Original Image", corrected_board)

#detect lines
gray = cv2.cvtColor(corrected_board, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 250, apertureSize=3)

#dilate the edges
kernel = np.ones((3, 3), np.uint8)

cv2.imshow("Canny Edges", edges)

lines = cv2.HoughLines(edges, 1, np.pi / 180, 150)  # Threshold 150, adjust as needed

# print the lines

if lines is not None:
    for rho, theta in lines[:, 0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
        pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
        cv2.line(corrected_board2, pt1, pt2, (0, 0, 255), 1)

cv2.imshow("Hough Lines", corrected_board2)




cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
import cv2
import numpy as np

# Make a copy of the corrected board
corrected_board2 = corrected_board.copy()

# Display the original board
cv2.imshow("board", corrected_board)


# Detect lines
gray = cv2.cvtColor(corrected_board, cv2.COLOR_BGR2GRAY)

#blur_7 = cv2.GaussianBlur(gray, (7, 7), 0)  # Large kernel

#gray = cv2.adaptiveThreshold(blur_7, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
#                                    cv2.THRESH_BINARY, 11, 2)



edges = cv2.Canny(gray, 50, 300, apertureSize=3)

# Dilate the edges (optional, helps connect small gaps)


# Show the Canny edges
cv2.imshow("Canny Edges", edges)

# Detect lines using Hough Transform
lines = cv2.HoughLines(edges, 1, np.pi / 180, 110)  # Threshold 150, adjust as needed

# Function to filter lines by minimum distance
def filter_lines(lines, min_dist):
    if not lines:
        return []
    filtered = [lines[0]]  # Start with the first line
    for rho, theta in lines[1:]:
        if abs(rho - filtered[-1][0]) >= min_dist:
            filtered.append((rho, theta))
    return filtered

# Separate horizontal and vertical lines
h_lines, v_lines = [], []
min_dist = 50  # Minimum distance between lines (adjust based on your image)
#min_dist = 0

if lines is not None:
    for rho, theta in lines[:, 0]:
        angle = theta * 180 / np.pi
        if 0 <= angle < 10 or 170 <= angle <= 180:  # Horizontal lines
            h_lines.append((rho, theta))
        elif 80 <= angle <= 100:  # Vertical lines
            v_lines.append((rho, theta))

    # Sort and filter lines to enforce minimum distance
    h_lines = sorted(h_lines, key=lambda x: x[0])
    v_lines = sorted(v_lines, key=lambda x: x[0])
    h_lines = filter_lines(h_lines, min_dist)[:100]  # Up to 9 horizontal lines for 8x8 grid
    v_lines = filter_lines(v_lines, min_dist)[:100]  # Up to 9 vertical lines for 8x8 grid

    # Draw horizontal lines (red)
    for rho, theta in h_lines:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
        pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
        cv2.line(corrected_board2, pt1, pt2, (0, 0, 255), 1)  # Red for horizontal

    # Draw vertical lines (green)
    for rho, theta in v_lines:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
        pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
        cv2.line(corrected_board2, pt1, pt2, (0, 255, 0), 1)  # Green for vertical

    print(f"Detected {len(h_lines)} horizontal lines and {len(v_lines)} vertical lines")

# Show the image with Hough lines
cv2.imshow("Hough Lines", corrected_board2)

cv2.waitKey(0)
cv2.destroyAllWindows()

Detected 14 horizontal lines and 8 vertical lines


representar em matriz ???

In [19]:
import os

#cut the corrected board by 35 on each side
corrected_board_cut = corrected_board[40:460, 40:460]

size = round((465-44) / 8)
#size = round((460-40) / 8)


# Create a temporary directory to save tiles if it doesn't exist
temp_dir = "temp_chess_tiles"
os.makedirs(temp_dir, exist_ok=True)

'''
#size = 50
for i in range(8):
    for j in range(8):
        cv2.rectangle(corrected_board_cut, (i*size, j*size), (i*size+size, j*size+size), (0, 255, 0), 1)
'''
# Extract and save each tile
for j in range(8):
    for i in range(8):
        # Extract the tile
        x1, y1 = i * size, j * size
        x2, y2 = x1 + size, y1 + size
        
        # Get the tile image
        tile = corrected_board_cut[y1:y2, x1:x2].copy()
        
        # Save the tile
        tile_filename = f"{temp_dir}/tile_{j}_{i}.jpg"
        cv2.imwrite(tile_filename, tile)
        
        # Draw rectangle on the original image for visualization
        cv2.rectangle(corrected_board_cut, (x1, y1), (x2, y2), (0, 255, 0), 1)

print(f"Saved 64 tiles to directory: {temp_dir}")

# Display the board with grid overlay
cv2.imshow("Board with grid", corrected_board_cut)
cv2.waitKey(0)
cv2.destroyAllWindows()



Saved 64 tiles to directory: temp_chess_tiles


In [20]:
def print_resized_up(name, img):
    resize_img = cv2.resize(img, (0, 0), fx=5.0, fy=5.0)
    cv2.imshow(name, resize_img)

#resize the tile
tile = cv2.imread("temp_chess_tiles/tile_0_0.jpg")
print_resized_up("Tile", tile)
cv2.waitKey(0)
cv2.destroyAllWindows()

Code to try for each tile

In [21]:
def check_B_in_W(img):
    # Display the original tile (assuming print_resized_up is a custom display function)
    #print_resized_up("Tile", img)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Binarize the image (black = 0, white = 255)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    #print_resized_up("Threshold", thresh)
    
    # Count black and white pixels
    black_pixels = np.sum(thresh == 0)  # Pixels with value 0
    white_pixels = np.sum(thresh == 255)  # Pixels with value 255
    
    # Print the counts for debugging
    #print(f"Black pixels: {black_pixels}, White pixels: {white_pixels}")
    
    # Return True if black pixels exceed white pixels
    return black_pixels > 650




def check_W_in_W(img):
    # Display the original tile
    #print_resized_up("Tile", img)

    # Convert to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #cv2.imshow("HSV", hsv)

    # Create a mask:
    # - S (saturation) < 82 (low saturation for white/light colors)
    # - V (value/brightness) < 101 (darker areas excluded)
    # Pixels within this range will be 255 (white), others 0 (black)
    upper_bound = (179, 255, 255)    # Minimum H, S, V
    lower_bound = (0, 82, 101)  # Maximum H, S, V
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    #print_resized_up("Mask", mask)

    # Count white pixels (255) and black pixels (0) in the mask
    white_pixels = np.sum(mask == 255)
    black_pixels = np.sum(mask == 0)
    #print(f"White pixels: {white_pixels}, Black pixels: {black_pixels}")

    # Optional: Display the HSV image again (not modified, so same as before)
    #print_resized_up("HSV2", hsv)

    # Return True if more white pixels than black (indicating a light area/piece)
    return white_pixels > 650




def check_W_in_B(img):
    # Display the original tile
    print_resized_up("Tile", img)

    # Convert to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    cv2.imshow("HSV", hsv)

    # Create a mask:
    # - S (saturation) < 82 (low saturation for white/light colors)
    # - V (value/brightness) < 101 (darker areas excluded)
    # Pixels within this range will be 255 (white), others 0 (black)
    upper_bound = (179, 255, 255)    # Minimum H, S, V
    lower_bound = (0, 49, 80)  # Maximum H, S, V
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    print_resized_up("Mask", mask)

    # Count white pixels (255) and black pixels (0) in the mask
    white_pixels = np.sum(mask == 255)
    black_pixels = np.sum(mask == 0)
    print(f"White pixels: {white_pixels}, Black pixels: {black_pixels}")

    # Optional: Display the HSV image again (not modified, so same as before)
    #print_resized_up("HSV2", hsv)

    # Return True if more white pixels than black (indicating a light area/piece)
    return white_pixels > 650





def check_B_in_B(img):
    # Display the original tile
    #print_resized_up("Tile", img)

    # Convert to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #cv2.imshow("HSV", hsv)

    # Create a mask:
    # - S (saturation) > 82 (low saturation for white/light colors)
    # - V (value/brightness) < 101 (darker areas excluded)
    # Pixels within this range will be 255 (white), others 0 (black)
    upper_bound = (179, 255, 30)    # Minimum H, S, V
    lower_bound = (0, 0, 0)  # Maximum H, S, V
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    #print_resized_up("Mask", mask)

    # Count white pixels (255) and black pixels (0) in the mask
    white_pixels = np.sum(mask == 255)
    black_pixels = np.sum(mask == 0)
    #print(f"White pixels: {white_pixels}, Black pixels: {black_pixels}")

    # Optional: Display the HSV image again (not modified, so same as before)
    #print_resized_up("HSV2", hsv)

    # Return True if more white pixels than black (indicating a light area/piece)
    return white_pixels > 650


'''
tile = cv2.imread("temp_chess_tiles/tile_1_4.jpg")
print(check_B_in_W(tile))

tile = cv2.imread("temp_chess_tiles/tile_3_0.jpg")
print(check_W_in_W(tile))
'''


tile = cv2.imread("temp_chess_tiles/tile_1_5.jpg")
print(check_W_in_B(tile))

'''
tile = cv2.imread("temp_chess_tiles/tile_1_3.jpg")
print(check_B_in_B(tile))
'''

cv2.waitKey(0)
cv2.destroyAllWindows()


White pixels: 165, Black pixels: 2644
False


Matrix representation

In [22]:

# Initialize counters for black and white pieces
blacks = 0
whites = 0

# Initialize the 8x8 matrix (0 for empty, 1 for piece)
chessboard_matrix = np.zeros((8, 8), dtype=int)

# Iterate over the 8x8 grid
for j in range(8):
    for i in range(8):
        # Load the tile image
        tile = cv2.imread(f"temp_chess_tiles/tile_{j}_{i}.jpg")
        
        if tile is None:
            print(f"Error: Could not load tile_{j}_{i}.jpg")
            continue

        # Check if the tile is black or white based on its position
        if (j + i) % 2 == 0:  # Black tile
            if check_B_in_B(tile):
                print(f"Tile {j}_{i} is a black piece.")
                blacks += 1
                chessboard_matrix[j, i] = 1  # Piece present
            elif check_W_in_B(tile):
                print(f"Tile {j}_{i} is a white piece.")
                whites += 1
                chessboard_matrix[j, i] = 1  # Piece present
        else:  # White tile
            if check_W_in_W(tile):
                print(f"Tile {j}_{i} is a white piece.")
                whites += 1
                chessboard_matrix[j, i] = 1  # Piece present
            elif check_B_in_W(tile):
                print(f"Tile {j}_{i} is a black piece.")
                blacks += 1
                chessboard_matrix[j, i] = 1  # Piece present

# Print the total counts
print(f"Total black pieces: {blacks}")
print(f"Total white pieces: {whites}")

# Print the matrix
print("Chessboard Matrix (1 for piece, 0 for empty):")
print(chessboard_matrix)


cv2.imshow("original",corrected_board)


cv2.waitKey(0)
cv2.destroyAllWindows()


White pixels: 0, Black pixels: 2809
Tile 0_1 is a black piece.
White pixels: 137, Black pixels: 2672
White pixels: 771, Black pixels: 2038
Tile 0_4 is a white piece.
Tile 0_5 is a black piece.
White pixels: 134, Black pixels: 2675
Tile 1_0 is a black piece.
White pixels: 276, Black pixels: 2533
Tile 1_2 is a black piece.
White pixels: 251, Black pixels: 2558
Tile 1_4 is a black piece.
White pixels: 165, Black pixels: 2644
Tile 1_6 is a black piece.
Tile 1_7 is a black piece.
White pixels: 148, Black pixels: 2661
Tile 2_1 is a black piece.
White pixels: 207, Black pixels: 2602
Tile 2_3 is a black piece.
White pixels: 218, Black pixels: 2591
Tile 2_5 is a black piece.
Tile 2_6 is a black piece.
Tile 2_7 is a black piece.
Tile 3_0 is a black piece.
White pixels: 561, Black pixels: 2248
Tile 3_2 is a black piece.
White pixels: 317, Black pixels: 2492
Tile 3_4 is a black piece.
White pixels: 314, Black pixels: 2495
Tile 3_6 is a white piece.
White pixels: 147, Black pixels: 2450
White pixel

Helper to decide the colour threshholds in hsv (não correr)

In [23]:
import cv2
import numpy as np

def nothing(x):
    pass

def hsv_color_picker(image):
    # Create a window
    cv2.namedWindow('HSV Color Picker')
    
    # Create trackbars for HSV values
    cv2.createTrackbar('H Min', 'HSV Color Picker', 0, 179, nothing)
    cv2.createTrackbar('S Min', 'HSV Color Picker', 0, 255, nothing)
    cv2.createTrackbar('V Min', 'HSV Color Picker', 0, 255, nothing)
    cv2.createTrackbar('H Max', 'HSV Color Picker', 179, 179, nothing)
    cv2.createTrackbar('S Max', 'HSV Color Picker', 255, 255, nothing)
    cv2.createTrackbar('V Max', 'HSV Color Picker', 255, 255, nothing)
    
    # Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    while True:
        # Get current positions of all trackbars
        h_min = cv2.getTrackbarPos('H Min', 'HSV Color Picker')
        s_min = cv2.getTrackbarPos('S Min', 'HSV Color Picker')
        v_min = cv2.getTrackbarPos('V Min', 'HSV Color Picker')
        h_max = cv2.getTrackbarPos('H Max', 'HSV Color Picker')
        s_max = cv2.getTrackbarPos('S Max', 'HSV Color Picker')
        v_max = cv2.getTrackbarPos('V Max', 'HSV Color Picker')
        
        # Set minimum and maximum HSV values
        lower = np.array([h_min, s_min, v_min])
        upper = np.array([h_max, s_max, v_max])
        
        # Create mask
        mask = cv2.inRange(hsv, lower, upper)
        
        # Apply mask to original image
        result = cv2.bitwise_and(image, image, mask=mask)
        
        # Show the images
        # Stack images horizontally for comparison
        stacked = np.hstack((image, result))
        cv2.imshow('HSV Color Picker', stacked)
        
        # Print current HSV values (useful for debugging)
        print(f"H_min: {h_min}, S_min: {s_min}, V_min: {v_min}, H_max: {h_max}, S_max: {s_max}, V_max: {v_max}", end="\r")
        
        # Break loop on ESC key
        k = cv2.waitKey(1) & 0xFF
        if k == 27:  # ESC key
            break
    
    # Print final values for easy copy-paste
    print(f"\nFinal values:")
    print(f"lower = np.array([{h_min}, {s_min}, {v_min}])")
    print(f"upper = np.array([{h_max}, {s_max}, {v_max}])")
    
    cv2.destroyAllWindows()
    return lower, upper

# Modified check_W_in_W function to use HSV color picker
def check_W_in_W(img):
    print_resized_up("Tile", img)
    
    # Use the HSV color picker to find the right thresholds
    lower, upper = hsv_color_picker(img)
    
    # Convert to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Create mask with the selected thresholds
    mask = cv2.inRange(hsv, lower, upper)
    
    # Apply mask
    result = cv2.bitwise_and(img, img, mask=mask)
    
    print_resized_up("HSV Filtered", result)
    
    # Count non-zero pixels in the mask
    white_piece_pixels = cv2.countNonZero(mask)
    print(f"White piece pixels: {white_piece_pixels}")
    
    return white_piece_pixels > 100  # Adjust threshold as needed

# Test the function
'''
tile = cv2.imread("temp_chess_tiles/tile_1_3.jpg")
if tile is not None:
    result = check_W_in_W(tile)
    print(f"White piece detected: {result}")
else:
    print("Could not load image")
    
cv2.waitKey(0)
cv2.destroyAllWindows()'
'''

'\ntile = cv2.imread("temp_chess_tiles/tile_1_3.jpg")\nif tile is not None:\n    result = check_W_in_W(tile)\n    print(f"White piece detected: {result}")\nelse:\n    print("Could not load image")\n    \ncv2.waitKey(0)\ncv2.destroyAllWindows()\'\n'

In [24]:
'''
import cv2
import numpy as np

# Load and preprocess the image
image = corrected_board  # Assuming corrected_board is already defined
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Detect lines using Hough Transform
lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

# Separate horizontal and vertical lines
h_lines, v_lines = [], []
if lines is not None:
    for rho, theta in lines[:, 0]:
        angle = theta * 180 / np.pi
        if 0 <= angle < 10 or 170 <= angle <= 180:  # Horizontal lines
            h_lines.append((rho, theta))
        elif 80 <= angle <= 100:  # Vertical lines
            v_lines.append((rho, theta))

# Function to filter lines by minimum distance
def filter_lines(lines, min_distance):
    if not lines:
        return []
    filtered = [lines[0]]  # Start with the first line
    for rho, theta in lines[1:]:
        # Check if the current rho is far enough from the last filtered line
        if abs(rho - filtered[-1][0]) >= min_distance:
            filtered.append((rho, theta))
    return filtered

# Sort and filter lines (e.g., min_distance = 20 pixels, adjust as needed)
min_distance = 30  # Adjust this value based on your image resolution
h_lines = sorted(h_lines, key=lambda x: x[0])
v_lines = sorted(v_lines, key=lambda x: x[0])
h_lines = filter_lines(h_lines, min_distance)[:9]  # Get up to 9 horizontal lines
v_lines = filter_lines(v_lines, min_distance)[:9]  # Get up to 9 vertical lines

# Check if we have enough lines
if len(h_lines) < 9 or len(v_lines) < 9:
    print(f"Warning: Found only {len(h_lines)} horizontal and {len(v_lines)} vertical lines.")
else:
    print(f"Found {len(h_lines)} horizontal and {len(v_lines)} vertical lines.")

# Create the 8x8 matrix
chessboard_matrix = np.zeros((8, 8), dtype=int)

# Draw the lines on the image
for rho, theta in h_lines:
    a, b = np.cos(theta), np.sin(theta)
    x0, y0 = a * rho, b * rho
    x1, y1 = int(x0 + 1000 * (-b)), int(y0 + 1000 * (a))
    x2, y2 = int(x0 - 1000 * (-b)), int(y0 - 1000 * (a))
    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red for horizontal

for rho, theta in v_lines:
    a, b = np.cos(theta), np.sin(theta)
    x0, y0 = a * rho, b * rho
    x1, y1 = int(x0 + 1000 * (-b)), int(y0 + 1000 * (a))
    x2, y2 = int(x0 - 1000 * (-b)), int(y0 - 1000 * (a))
    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green for vertical

# Display the image
cv2.imshow('Detected Lines', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Optional: Print the matrix (you can populate it later)
print("Chessboard Matrix (placeholder):")
print(chessboard_matrix)'
'''

'\nimport cv2\nimport numpy as np\n\n# Load and preprocess the image\nimage = corrected_board  # Assuming corrected_board is already defined\ngray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\ngray = cv2.GaussianBlur(gray, (5, 5), 0)\nedges = cv2.Canny(gray, 50, 150, apertureSize=3)\n\n# Detect lines using Hough Transform\nlines = cv2.HoughLines(edges, 1, np.pi / 180, 200)\n\n# Separate horizontal and vertical lines\nh_lines, v_lines = [], []\nif lines is not None:\n    for rho, theta in lines[:, 0]:\n        angle = theta * 180 / np.pi\n        if 0 <= angle < 10 or 170 <= angle <= 180:  # Horizontal lines\n            h_lines.append((rho, theta))\n        elif 80 <= angle <= 100:  # Vertical lines\n            v_lines.append((rho, theta))\n\n# Function to filter lines by minimum distance\ndef filter_lines(lines, min_distance):\n    if not lines:\n        return []\n    filtered = [lines[0]]  # Start with the first line\n    for rho, theta in lines[1:]:\n        # Check if the current rh

In [25]:

'''

#find the corners

corrected_board2 = corrected_board.copy()

gray = cv2.cvtColor(corrected_board2, cv2.COLOR_BGR2GRAY)

corners = cv2.goodFeaturesToTrack(gray, 130, 0.05, 10)
corners = np.intp(corners)

for corner in corners:
    x, y = corner.ravel()
    cv2.circle(corrected_board2, (x, y), 3, 255, -1)

cv2.imshow("Corners", corrected_board2)
cv2.waitKey(0)
cv2.destroyAllWindows()

'''


'\n\n#find the corners\n\ncorrected_board2 = corrected_board.copy()\n\ngray = cv2.cvtColor(corrected_board2, cv2.COLOR_BGR2GRAY)\n\ncorners = cv2.goodFeaturesToTrack(gray, 130, 0.05, 10)\ncorners = np.intp(corners)\n\nfor corner in corners:\n    x, y = corner.ravel()\n    cv2.circle(corrected_board2, (x, y), 3, 255, -1)\n\ncv2.imshow("Corners", corrected_board2)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n\n'